In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd "drive/MyDrive/Colab Notebooks/trolleyproblem"

/content/drive/MyDrive/Colab Notebooks/trolleyproblem


# Welcome to your final excercise

Today, you'll explore the **trolley problem**, a scenario in which an autonomous vehicle must decide whom to prioritize in a critical situation

The exercise is structured as follows:

1. What is the trolley problem?
2. Fill out the MIT-Questionnaire
3. Code walkthrough
4. Challenge the AI
5. Report your findings and discuss


# What is the trolley problem?

The trolley problem is a traditional ethical thought experiment that poses inquiries about moral dilemmas and the decisions individuals make in such situations. It is commonly utilized in philosophical and ethical discussions to examine moral principles and decision-making. The scenario typically unfolds as follows:

**Setting:** Imagine there is a trolley (a runaway tram) headed down a track towards five people tied up and unable to move. They are in immediate danger of being hit and killed by the trolley.

**Dilemma:** You are standing next to a lever that can switch the trolley onto a different track. On this alternate track, there is one person tied up, and you have the option to pull the lever to divert the trolley.

**The moral question:** You are faced with a moral choice. Do you pull the lever to save the five people but result in the death of one person, or do you do nothing, allowing the trolley to continue on its current course, leading to the death of five people?


**This dilemma can be translated into the context of autonomous driving, where it is not the individual but the AI that is faced with the choice of whom to prioritize.**

![moral_machine](Moral_Machine.png)

### In this exercise today we will break down to a much simpler choice of left or right.

# MIT Moral Machine

The **Moral Machine** was developed by the MIT to gather human perspectives on moral decisions made by AI. You will be faced with a number of moral dilemmas, where a driverless car must choose the lesser of two evils, such as killing two passengers or five pedestrians.

## Your Task:

1. Fill out the Moral Machine Questionnaire, which can be found [here](https://www.moralmachine.net/).
    1. Click on "Start Judging".
    2. Decide what the self driving car should do (click on "Show Description for more information").
    3. Check your results in the end.
2. Report your findings and thoughts in this notebook

Write a five sentence summary of your thoughts on the ethical dilemmas you faced.  
Think about your descisions: Were they easy or hard? What guidelines should AI stick to when facing such descisions?

.  
.  
.  
.  
.  
(double-click into this cell to edit; shift+enter to render)  

# Simulation

For this exercise we will build a simple simulation to test AI decision making in a simplified scenario. In our case the AI must make a choice between left or right. Internally the decision is based upon 50 risk values (25 for each side), which are numbers between 0.0 and 1.0, where 0.0 corresponds to no risk and 1.0 corresponds to high risk. Clearly, the AI is programmed to opt for the side that, on the whole, presents the least amount of risk.

***We will now walk you through the code of this simulation***

The objective here is to ensure your grasp of the fundamentals of the AI code. In this walkthrough, we'll demonstrate various methods to influence the AI's decision-making. By modifying just a few parameters, we can alter the AI's outcome, such as consistently favoring the left side. It is crucial for you to comprehend the functioning of these parameters, as you will be adjusting them when engaging with this simulation later.

## Walkthrough

Let´s start our walkthrough by discussing what we need to build a simple AI that is capable of deciding between left and right based on a list of numbers.

1. **Identify the problem**  
The decision between left and right is a binary classification problem. Binary means we have two classes (left, right). Our ML-Algorithm should output a probability for each class, which must add up to 1.0.  
**Our output could look something like this:**
Left: 0.3542; Right: 0.6458  
In this example our AI is almost 65% sure, that running over the person on the right is the best choice.

2. **Choosing a ML-Algorithm**  
There are lots of different ML-Algorithms that can handle binary classification problems very well. In our case we went for a Neural Network since it´s main process is quite easy to understand. We will talk about the specifics later.

3. **Data**  
You already are aware of the fact, that NNs need training data that contain labels. If you are not familiar with the way NNs work and are trained, please refer to one of the earlier lectures and exercises.
To provide training data you usually have to collect data from real world scenarios and label them by hand. In our simulation, we created a synthetic dataset, that contains random values, which are generated in a specific way to automatically fit our prupose.  
We won´t discuss the creation of our synthetic dataset in any more detail. If you are curious, try to make sense of the create_data function in data_provider.py

4. **Code**  
We now have to write our ideas in code.

## Walkthrough Step 1 - Defining the Neural Network

We already discussed what our output should look like. So let´s start with the input.

We have a list of risk values that contains the same number of values for each side. For 25 risk values and two sides that makes a total of 50 risk values, which serve as our input data.

Our goal is now to feed these 50 values directly into the first layer of our Neural-Network.

Using pytorch, defining our model is made very easy. The code for our fist layer looks like this:

```python
  self.layer1 = torch.nn.Sequential(
            torch.nn.Linear(in_features=num_risk_values*2, out_features=32, bias=False),
            torch.nn.ReLU(inplace=True)
        )
```

The out_features of one linear layer always presets the in_features of the next linear layer. Therefor our next layer must have 32 in_features. This layer also serves already as our output layer, because there is no need for the NN to be any bigger. *(Try to keep the amount parameters as low as possible in order to avoid overfitting)*

Since we already discussed, that we want two probabilties as our output, we need two output features.
To activate our output we make use of the softmax funtion, which you can find in almost every output layer of any classification problem, because it returns a distribution of probabilities on as many output features as you like, that always adds up to 1.0

*Say for example the last linear layer return something like: (-0.3561, 0.36224) -> Softmax then could return something like (0.435, 0.565)

```python
  self.output = torch.nn.Sequential(
            torch.nn.Linear(in_features=32, out_features=2, bias=False),
            torch.nn.Softmax()
        )
```

**We need to tell our NN how to process any inputs**

Therefore every NN needs a forward function, which looks like this for input x:

```python
      def forward(self, x):
        output = self.layer1(x)
        output = self.output(output)
        return output
```

We simply tell our model to process any input *x* by applying layer1 and the applying the output layer. The result of the output layer will then be returned.

Since NN´s in pytorch are defined as classes, our full model definition code looks like this:

```python
    class TrolleyClassifier(torch.nn.Module):
        def __init__(self, num_risk_values):
            super().__init__()
            self.layer1 = torch.nn.Sequential(
                torch.nn.Linear(in_features=num_risk_values*2, out_features=32, bias=False),
                torch.nn.ReLU(inplace=True)
            )
            self.output = torch.nn.Sequential(
                torch.nn.Linear(in_features=32, out_features=2, bias=False),
                torch.nn.Softmax()
            )

        def forward(self, x):
            output = self.layer1(x)
            output = self.output(output)
            return output
```

That´s it!

## Walkthrough Step 2 - Training of NN

Let´s talk about training an AI.

In every training step you do the following:

1. Feed data into the network
2. Compare output to label
3. Calculate error
4. Backward error through the network (adjusting weights)

If you repeat this for every training sample you have, you completed one epoch. Usually NN´s are trained for multiple epochs until the accuracy more or less converges.

In order to make the training more effective in terms of speed and accuracy a network is usually trained in batches of training data, which is just subset of entire dataset. Feel free to research a bit more about why batches are useful in terms of speed and better training results.

**Wrapping up:**

Let´s say we have a total of 500 training samples. We set a batch_size of 25 and want to train for 5 epochs. That means for every epoch we are doing 500/25 = 20 training steps.

You might remember Optimizers and Loss-Functions. We won´t get into the details of those functions in this exercise, except to say that optimizers can adjust the way a network learns over time and loss functions are used to compare training outputs to labels.

### Training Loop

Our training is done using a simple for loop in python that iterates over the epochs and the batches. To do that we had to define a dataset class and build a data loader, which dynamically loads the batches. (It´s all very basic and not very interesting)

**Optimizer & Loss**

```python
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_fn = torch.nn.BCELoss()
```

We used the Adam-Optimizer with learning rate of 0.01 and the BinaryCrossEntropyLoss.

```python
 1   for i in range(epochs):
 2       if(verbose):
 3           print(f"Epoch: {i+1} -----------------")
 4       model.train()
 5       for batch_number, data in enumerate(trainloader):
 6           risk_values, labels = data
 7           pred = model(risk_values)
 8           loss = loss_fn(pred, labels)
 9           loss.backward()
 10          optimizer.step()
 11          optimizer.zero_grad()
 12           if(verbose):
 13               print(f"[{batch_number+1}/{len(trainloader)}] - Loss: {loss}")
```

The train loop itself might seem complicated at first, but it´s actually quite simple. We use a stacked for loop to iterate over epochs and batches. In order to be able to change the weights in a model it has to be set in train-mode. After that we grab the data and labels of every batch from the training data (line 6), predict them on the NN (line 7), calculate the loss (line 8), perform the backward pass (line 9), notify the optimizer (line 10) and finally reset the gradients of the model parameters to zero.


# Challenge the AI

The goal of this exercise is to experience the AI´s decision-making. The following user-interface will allow you to create personas which are than automatically translated into risk values, which are than fed into the network.

## Parameters

Think back to the MIT-Questionnaire, where you decided what to do. The UI allows you to favor certain attributes of your personas. For example protect women over men or young people over old people.

For each persona parameter you can set a weight between 0.0 and 1.0.

It´s highly recommended to play around with different parameter settings to force interesting decisions from the AI.
Try to give the AI complicated choices and see how it handles it.

In [3]:
import ui_utils

ui_utils.persona_ui()

Now this is all fine! But what can we learn from this? Is it really you who made the descision? What parts of the descision weren´t in your hands? Could you come to terms witht he fact your Weigths might have caused an antonomous vehicle to kill someone. Would you rather have the AI decide? But is that really more ethical?

Please write down your thoughts and findings in the cell below. Think about the different scenarios you encountered. Think about the different parameters you adjusted and why adjusted them the way you did? We are also interested to here your ethical point of view on the entire trolley problem.

Report your findings here. Write at least 15 sentences!

.  
.  
.  
.  
.  
.  
.  
.  
.  
.  
.  
(double-click into this cell to edit; shift+enter to render)